- take the best performing model and do some clustering to identify the races it performs well on.

In [1]:
import sys
sys.path.append('../src')

import mlflow
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression # type: ignore
from sklearn.model_selection import GridSearchCV # type: ignore

import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler, SMOTE
import pickle

from functions import *

from data_cleaning import DataCleaning

import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
data = pd.read_csv('../data/turf.csv')
data.head()

,Course,Course Direction,Distance (y),Race Type,Class,Going,Race Date,Race Time,Racecard No.,Draw,Draw IV,Position,Horse,DTN,DTW,Jockey,Jockey Claim,Trainer,Age,Weight (pounds),Penalty,Weight Rank,DSLR,Equip,Sex Abbrev,Favourite,BF Decimal SP,"Won (1=Won, 0=Lost)","Place (1=Placed, 0=UnPlaced)",PRC Average,PRC Last Run,PRC 2nd Last Run,PRC 3rd Last Run,Pace Abbrev,Pace Rating,Pace Rating Rank,OR,OR Rank,Tissue Odds Text,Tissue Odds Decimal,Tissue Rating,Tissue Rating Rank,TIS+PFR,PFR,PFR Rank,Main,Main Rank,Main Stats,Main+Stats Rank,Stats,Stats Rank,Trn Stats,Trainer Stats Rank,Jky Stats,Jockey Stats Rank,TrnJky Stats,Trainer/Jky Stats Rank,Hrs Stats,Horse Stats Rank,UR1,UR1 Rank,UR2,UR2 Rank,UR3,UR3 Rank,UR4,UR4 Rank,HA Career Speed Rating,HA Career Speed Rating Rank,HA Last 1 Year Speed Rating,HA Last 1 Year Speed Rating Rank,MR Career Speed Rating,MR Career Speed Rating Rank,MR Last 1 Year Speed Rating,MR Last 1 Year Speed Rating Rank,MR Last 3 Runs Speed Rating,MR Last 3 Runs Speed Rating Rank,LTO Speed Rating,LTO Speed Rating Rank,2nd LTO Speed Rating,2nd LTO Speed Rating Rank,3rd LTO Speed Rating,3rd LTO Speed Rating Rank,4th LTOt Speed Rating,4th LTO Speed Rating Rank,Horse Time In Seconds,dyHA,dyLTO,dyAVG,dyAVG(x),dyMR,dyMR(x),dy2HA,Runs Before,Won Before,Distance In Yards LTO (Same Code),Official Rating LTO (Same Code),FLAT TURF Runs Before,NHF Wins Before,Distance To Winner LTO,User Rating 1 LTO,User Rating 2 LTO,User Rating 3 LTO,User Rating 4 LTO,Speed Rating LTO,PRC Average LTO,PRC Last Run LTO,PRC 2nd Last Run LTO,PRC 3rd Last Run LTO,Today's Going Wins,Today's Going Places,Today's Going Runs,Today's Going PRB,Today's Going PRB^2,Today's Going 50% SR,Today's Distance Wins,Today's Distance Places,Today's Distance Runs,Today's Distance PRB,Today's Distance PRB^2,Today's Distance 50% SR,Today's Class Wins,Today's Class Places,Today's Class Runs,Today's Class PRB,Today's Class PRB^2,Today's Class 50% SR,Today's Course Wins,Today's Course Places,Today's Course Runs,Today's Course PRB,Today's Course PRB^2,Today's Course 50% SR,Pace Abbreviation LTO (Same Code)
0,Ascot,RH,1100,Turf,A,Good,05/10/2024,13:50,1,5,1.05,6.0,Democracy Dilemma,7.0,16.049999,Jamie Spencer,0,Robert Cowell,4,132,0,1,14.0,NaN,g,NaN,6.79,0,0,94,90,97,94,L,12,1,106,1,9/2,5.5,36,3,80,44,3,334,3,663,3,329,4,83,4,86,5,82,6,78,3,215,3,NaN,1,NaN,1,NaN,1,81,2,81,2,65,5,75,1,79,1,81,2,79,3,75,1,67,2,64.669998,81.0,81.0,62.0,78.0,64.0,79.0,79.0,29,7,1134,106.0,20,0,1.65,147.0,NaN,NaN,NaN,81.0,94.0,97.0,94.0,91.0,1,0,3,57.41,43.31,66.67,7,5,24,67.26,53.97,70.83,1,0,6,48.46,32.13,50.00,0,0,1,21.74,4.73,0.00,L
1,Ascot,RH,1100,Turf,A,Good,05/10/2024,13:50,2,3,0.67,5.0,Go Athletico,6.0,9.050000,D Egan,0,Adrian McGuinness,6,129,0,2,6.0,NaN,g,NaN,10.38,0,0,88,84,86,94,P,4,3,104,2,9/2,5.5,35,4,64,29,5,317,4,635,5,318,5,78,6,76,6,88,3,76,4,103,5,NaN,1,NaN,1,NaN,1,78,5,71,6,72,2,70,4,70,4,65,6,70,5,71,4,0,4,62.919998,78.0,65.0,72.0,68.0,72.0,70.0,75.0,7,2,1320,104.0,7,0,3.35,93.0,NaN,NaN,NaN,65.0,93.0,86.0,94.0,98.0,1,1,2,92.86,86.73,50.00,1,2,4,73.23,60.65,25.00,2,3,7,71.84,61.09,28.57,0,0,0,NaN,NaN,NaN,P
2,Ascot,RH,1100,Turf,A,Good,05/10/2024,13:50,3,1,1.24,1.0,Rumstar,0.0,0.000000,Hector Crouch,0,Jonathan Portman,4,129,0,2,21.0,NaN,g,NaN,7.08,1,0,87,93,76,91,P,4,3,101,5,14/1,15.0,13,6,32,19,6,281,6,579,6,298,6,80,5,91,2,83,5,44,6,93,6,NaN,1,NaN,1,NaN,1,78,5,78,5,69,4,70,4,70,4,70,4,66,6,72,3,64,3,60.660000,78.0,70.0,61.0,69.0,66.0,70.0,76.0,19,3,1243,101.0,18,0,0.70,29.0,NaN,NaN,NaN,70.0,78.0,76.0,91.0,68.0,0,0,0,NaN,NaN,NaN,2,1,6,65.69,56.17,66.67,1,0,9,50.49,31.89,44.44,0,0,3,34.72,18.58,33.33,P
3,Ascot,RH,1100,Turf,A,Good,05/10/2024,13:50,4,4,1.43,2.0,Beautiful Diamond,2.0,2.000000,Clifford Lee,0,K R Burke,3,127,0,4,20.0,NaN,f,3.0,4.05,0,1,95,91,96,97,P,6,2,104,2,7/2,4.5,44,2,90,46,2,342,2,683,2,341,3,91,3,87,4,96,2,67,5,231,2,NaN,1,NaN,1,NaN,1,80,4,80,4,74,1,75,1,75,2,74,3,80,2,75,1,75,1,61.160000,80.0,74.0,72.0,76.0,74.0,75.0,78.0,9,3,1100,104.0,9,0,4.20,31.0

In [3]:
data['BF Decimal SP1'] = data['BF Decimal SP']


In [4]:
bins = [800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600,
        2800, 3000, 3200, 3400, 3600, 3800, 4000]

data['distance_bucket'] = pd.cut(data['Distance (y)'], bins = bins,
                               labels=['800-1000', '1000-1200', '1200-1400', '1400-1600', '1600-1800', '1800-2000',
                                       '2000-2200', '2200-2400', '2400-2600', '2600-2800', '2800-3000',
                                       '3000-3200', '3200-3400', '3400-3600', '3600-3800', '3800-4000'])

Create a df with 5 independent features and 'Won (1=Won, 0=Lost)' as the dependent variable

In [ ]:
df = data[['Race Date','Race Time', 'Course', 'Horse','distance_bucket', 'Won (1=Won, 0=Lost)', 'BF Decimal SP1', 'UR1 Rank', 
           'Main Rank', 'Main+Stats Rank', 'PFR Rank']]

In [6]:
df.dtypes

Race Time                object
Course                   object
Horse                    object
distance_bucket        category
Won (1=Won, 0=Lost)       int64
BF Decimal SP1          float64
UR1 Rank                  int64
Main Rank                 int64
Main+Stats Rank           int64
PFR Rank                  int64
dtype: object

In [7]:
df.head(5)

,Race Time,Course,Horse,distance_bucket,"Won (1=Won, 0=Lost)",BF Decimal SP1,UR1 Rank,Main Rank,Main+Stats Rank,PFR Rank
0,13:50,Ascot,Democracy Dilemma,1000-1200,0,6.79,3,3,3,3
1,13:50,Ascot,Go Athletico,1000-1200,0,10.38,5,4,5,5
2,13:50,Ascot,Rumstar,1000-1200,1,7.08,6,6,6,6
3,13:50,Ascot,Beautiful Diamond,1000-1200,0,4.05,2,2,2,2
4,13:50,Ascot,Miaharris,1000-1200,0,13.50,4,5,4,4


In [8]:
# Get all numeric columns
numeric_cols = ['UR1 Rank', 'Main Rank', 'Main+Stats Rank', 'PFR Rank']

# Replace NaN in all numeric columns
cleaned_df = DataCleaning.replace_nan(df, numeric_cols)


In [9]:
normalized_df = DataCleaning.normalize_columns(cleaned_df, numeric_cols)

normalized_df.head(5)

,Race Time,Course,Horse,distance_bucket,"Won (1=Won, 0=Lost)",BF Decimal SP1,UR1 Rank,Main Rank,Main+Stats Rank,PFR Rank
0,13:50,Ascot,Democracy Dilemma,1000-1200,0,6.79,0.064516,0.060606,0.060606,0.060606
1,13:50,Ascot,Go Athletico,1000-1200,0,10.38,0.129032,0.090909,0.121212,0.121212
2,13:50,Ascot,Rumstar,1000-1200,1,7.08,0.161290,0.151515,0.151515,0.151515
3,13:50,Ascot,Beautiful Diamond,1000-1200,0,4.05,0.032258,0.030303,0.030303,0.030303
4,13:50,Ascot,Miaharris,1000-1200,0,13.50,0.096774,0.121212,0.090909,0.090909


In [10]:
target_col = 'Won (1=Won, 0=Lost)'

feature_cols = [col for col in numeric_cols if col != target_col]

train_data, test_data = DataCleaning.split_data(df = normalized_df)

X_train = train_data[feature_cols]
y_train = train_data[target_col]

X_test = test_data[feature_cols]
y_test = test_data[target_col]


In [11]:
X_train.head()

,UR1 Rank,Main Rank,Main+Stats Rank,PFR Rank
0,0.064516,0.060606,0.060606,0.060606
1,0.129032,0.090909,0.121212,0.121212
2,0.161290,0.151515,0.151515,0.151515
3,0.032258,0.030303,0.030303,0.030303
4,0.096774,0.121212,0.090909,0.090909


In [12]:
X_test.head()

,UR1 Rank,Main Rank,Main+Stats Rank,PFR Rank
316,0.000000,0.030303,0.030303,0.030303
317,0.161290,0.000000,0.000000,0.060606
318,0.064516,0.121212,0.121212,0.242424
319,0.129032,0.242424,0.212121,0.121212
320,0.032258,0.060606,0.151515,0.090909


In [13]:
# count of unique races
unique_race_count = test_data['Race Time'].nunique()

print(f'Count of unique races in the test data is {unique_race_count}')

Count of unique races in the test data is 207


# Use grid search to idenify the best hyperparameters for the model

In [14]:
est_data_ = test_data.copy()

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

y_pred1 = best_model.predict(X_test)
est_data_['model_preds'] = y_pred1

print_metrics(y_test, y_pred1)




Confusion Matrix: 
[[11449    76]
 [ 1468     7]]
Accuracy: 0.8812307692307693
Precision: 0.08433734939759036
Recall: 0.0047457627118644066
F1 score: 0.008985879332477536
ROC AUC score: 0.4990757013125482


In [15]:
profit_calculation(est_data_)

Total number of bets: 83
Total return from betting £1.00 on each prediction where model_preds == 1: £-27.92
Return per pound invested: £-0.34
Model accuracy: 8.43%


# Run the data through a random forest classifier.

In [16]:
test_data1 = test_data.copy()

# Instantiate and train the Random Forest classifier
params = {
    'n_estimators' : 100,
    'random_state' : 42
}
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    rf_classifier = RandomForestClassifier(**params)
    rf_classifier.fit(X_train, y_train)

    # Log the trained model
    mlflow.sklearn.log_model(rf_classifier, "random_forest_model")

    # Make predictions on the test set
    y_pred = rf_classifier.predict(X_test)

    test_data1['model_preds'] = y_pred

    # Evaluate the model
    # use the eval method
    print_metrics(y_test, y_pred)
    mlflow.end_run()

with open('../models/base_model.pkl', 'wb') as f:
    pickle.dump(rf_classifier, f)

2025/09/10 15:47:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/10 15:47:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Confusion Matrix: 
[[11440    85]
 [ 1468     7]]
Accuracy: 0.8805384615384615
Precision: 0.07608695652173914
Recall: 0.0047457627118644066
F1 score: 0.008934269304403318
ROC AUC score: 0.49868524578109485


In [17]:
profit_calculation(test_data1)

Total number of bets: 92
Total return from betting £1.00 on each prediction where model_preds == 1: £-36.92
Return per pound invested: £-0.40
Model accuracy: 7.61%


In [18]:
test_data2 = test_data.copy()


# Example class weights for an imbalanced dataset
class_weights = {0: 1, 1: 5}

# Initialize and fit the RandomForest model
model = RandomForestClassifier(class_weight=class_weights, max_depth=10, min_samples_leaf=5)
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred1 = model.predict(X_test)

test_data2['model_preds'] = y_pred1

print_metrics(y_test, y_pred1)



Confusion Matrix: 
[[9759 1766]
 [ 931  544]]
Accuracy: 0.7925384615384615
Precision: 0.2354978354978355
Recall: 0.3688135593220339
F1 score: 0.2874504623513871
ROC AUC score: 0.6077907276002794


In [19]:
profit_calculation(test_data2)

Total number of bets: 2310
Total return from betting £1.00 on each prediction where model_preds == 1: £116.99
Return per pound invested: £0.05
Model accuracy: 23.55%


In [47]:
test_data2.head(50)

,Race Time,Course,Horse,distance_bucket,"Won (1=Won, 0=Lost)",BF Decimal SP1,UR1 Rank,Main Rank,Main+Stats Rank,PFR Rank,model_preds
316,17:15,Ascot,Aramram,1000-1200,0,3.55,0.000000,0.030303,0.030303,0.030303,1
317,17:15,Ascot,Tropical Storm,1000-1200,0,10.12,0.161290,0.000000,0.000000,0.060606,1
318,17:15,Ascot,Cover Up,1000-1200,1,4.80,0.064516,0.121212,0.121212,0.242424,0
319,17:15,Ascot,Roman Dragon,1000-1200,0,109.38,0.129032,0.242424,0.212121,0.121212,0
320,17:15,Ascot,Vintage Clarets,1000-1200,0,19.23,0.032258,0.060606,0.151515,0.090909,0
321,17:15,Ascot,Azure Angel,1000-1200,0,21.48,0.193548,0.060606,0.060606,0.000000,0
322,17:15,Ascot,Toca Madera,1000-1200,0,18.50,0.096774,0.151515,0.242424,0.212121,0
323,17:15,Ascot,Emperor Spirit,1000-1200,0,20.89,0.290323,0.181818,0.090909,0.151515,0
324,17:15,Ascot,Woolhampton,1000-1200,0,25.00,0.258065,0.272727,0.272727,0.333333,0
325,17:15,Ascot,Glamorous Breeze,1000-1200,0,22.00,0.354839,0.333333,0.181818,0.181818,0


# Instantiate and train the Gradient Boosting classifier

In [ ]:
test_data_gbm = test_data.copy()

# Instantiate the Gradient Boosting Classifier
gbm = GradientBoostingClassifier()

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Instantiate the GridSearchCV object
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=5, scoring='f1', n_jobs=-1, verbose=2)

# Fit Grid Search
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred_gbm = best_model.predict(X_test)

test_data_gbm['model_preds'] = y_pred_gbm


Fitting 5 folds for each of 243 candidates, totalling 1215 fits
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.9s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.0s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.0s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.0s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.0s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.0s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.1s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100; 

In [39]:
print_metrics(y_test, y_pred_gbm)


Confusion Matrix: 
[[11466    59]
 [ 1471     4]]
Accuracy: 0.8823076923076923
Precision: 0.06349206349206349
Recall: 0.002711864406779661
F1 score: 0.005201560468140442
ROC AUC score: 0.49879627927497333


In [22]:
profit_calculation(test_data_gbm)

Total number of bets: 63
Total return from betting £1.00 on each prediction where model_preds == 1: £15.21
Return per pound invested: £0.24
Model accuracy: 6.35%


In [23]:
# Instantiate and train the Gradient Boosting classifier

test_data3 = test_data.copy()

gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_gb = gb_classifier.predict(X_test)

test_data3['model_preds'] = y_pred_gb

print_metrics(y_test, y_pred_gb)


Confusion Matrix: 
[[11521     4]
 [ 1475     0]]
Accuracy: 0.8862307692307693
Precision: 0.0
Recall: 0.0
F1 score: 0.0
ROC AUC score: 0.49982646420824295


In [24]:
profit_calculation(test_data3)

Total number of bets: 4
Total return from betting £1.00 on each prediction where model_preds == 1: £-4.00
Return per pound invested: £-1.00
Model accuracy: 0.00%


In [25]:
test_data4 = test_data.copy()

model = XGBClassifier(reg_lambda=1, reg_alpha=0)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_xgb = model.predict(X_test)

test_data4['model_preds'] = y_pred_xgb

print_metrics(y_test, y_pred_gb)


Confusion Matrix: 
[[11521     4]
 [ 1475     0]]
Accuracy: 0.8862307692307693
Precision: 0.0
Recall: 0.0
F1 score: 0.0
ROC AUC score: 0.49982646420824295


In [26]:
profit_calculation(test_data4)

Total number of bets: 5
Total return from betting £1.00 on each prediction where model_preds == 1: £-5.00
Return per pound invested: £-1.00
Model accuracy: 0.00%


# Look at utilizing Random Oversampling on the dataset. This is due to the imbalanced nature of the target column.

In [27]:
# Apply Random Oversampling
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

print(f"Length of  X_train: {len(X_train)}\nLength of y_train: {len(y_train)}")
print(f"\nLength of  X_resampled: {len(X_resampled)}\nLength of y_resampled: {len(y_resampled)}")

Length of  X_train: 52680
Length of y_train: 52680

Length of  X_resampled: 93520
Length of y_resampled: 93520


In [28]:
# Instantiate and train the Gradient Boosting classifier
test_data5 = test_data.copy()

gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_classifier.fit(X_resampled, y_resampled)

# Make predictions on the test set
y_pred_gbx = gb_classifier.predict(X_test)

test_data5['model_preds'] = y_pred_gbx

print_metrics(y_test, y_pred_gbx)


Confusion Matrix: 
[[6815 4710]
 [ 476  999]]
Accuracy: 0.6010769230769231
Precision: 0.17498686284813453
Recall: 0.6772881355932203
F1 score: 0.27811804008908686
ROC AUC score: 0.6343056730026839


In [29]:
profit_calculation(test_data5)

Total number of bets: 5709
Total return from betting £1.00 on each prediction where model_preds == 1: £109.27
Return per pound invested: £0.02
Model accuracy: 17.50%


# Try an approach using smote

In [30]:
test_data6 = test_data.copy()

smote= SMOTE(random_state=42)

X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_classifier.fit(X_train_smote, y_train_smote)

# Make predictions on the test set
y_pred_gbs = gb_classifier.predict(X_test)

test_data6['model_preds'] = y_pred_gbs

# Evaluate the model
# use the eval method
print_metrics(y_test, y_pred)


Confusion Matrix: 
[[11440    85]
 [ 1468     7]]
Accuracy: 0.8805384615384615
Precision: 0.07608695652173914
Recall: 0.0047457627118644066
F1 score: 0.008934269304403318
ROC AUC score: 0.49868524578109485


In [31]:
profit_calculation(test_data6)

Total number of bets: 4532
Total return from betting £1.00 on each prediction where model_preds == 1: £219.63
Return per pound invested: £0.05
Model accuracy: 19.17%


In [32]:
test_data7 = test_data.copy()

X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

rf_classifier = RandomForestClassifier(**params)
rf_classifier.fit(X_train_smote, y_train_smote)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

test_data7['model_preds'] = y_pred

# Evaluate the model
# use the eval method
print_metrics(y_test, y_pred)


Confusion Matrix: 
[[8430 3095]
 [ 734  741]]
Accuracy: 0.7054615384615385
Precision: 0.19316996871741396
Recall: 0.5023728813559322
F1 score: 0.2790434946337789
ROC AUC score: 0.6169131218059487


In [33]:
profit_calculation(test_data7)

Total number of bets: 3836
Total return from betting £1.00 on each prediction where model_preds == 1: £114.85
Return per pound invested: £0.03
Model accuracy: 19.32%


In [34]:
test_data8 = test_data.copy()

# Assume X_train, X_test, y_train, y_test are already defined
# Setting class_weight to 'balanced' makes the algorithm cost-sensitive
model = RandomForestClassifier(class_weight='balanced')

# Training the model
model.fit(X_train, y_train)

# Predictions
y_predx = model.predict(X_test)

test_data8['model_preds'] = y_predx

# Evaluation
print_metrics(y_test, y_predx)


Confusion Matrix: 
[[7893 3632]
 [ 675  800]]
Accuracy: 0.6686923076923077
Precision: 0.18050541516245489
Recall: 0.5423728813559322
F1 score: 0.2708650753343491
ROC AUC score: 0.6136159417625647


In [35]:
profit_calculation(test_data8)

Total number of bets: 4432
Total return from betting £1.00 on each prediction where model_preds == 1: £-52.43
Return per pound invested: £-0.01
Model accuracy: 18.05%


In [36]:
test_data9 = test_data.copy()

# Define cost matrix
C_FN = 5  # Cost of false negative
C_FP = 1  # Cost of false positive

# Assuming y_train has values 0 and 1
weights = np.ones(y_train.shape[0])
weights[y_train == 1] = C_FN
weights[y_train == 0] = C_FP

# Train logistic regression model with custom weights
model = LogisticRegression()
model.fit(X_train, y_train, sample_weight=weights)

# Predictions
y_predz = model.predict(X_test)

test_data9['model_preds'] = y_predz

# Evaluation
print_metrics(y_test, y_predz)



Confusion Matrix: 
[[9414 2111]
 [ 862  613]]
Accuracy: 0.7713076923076924
Precision: 0.2250367107195301
Recall: 0.41559322033898305
F1 score: 0.29197427959037864
ROC AUC score: 0.6162130960697084


In [37]:
profit_calculation(test_data9)

Total number of bets: 2724
Total return from betting £1.00 on each prediction where model_preds == 1: £138.94
Return per pound invested: £0.05
Model accuracy: 22.50%


# Try a tensorflow model

In [38]:
test_data10 = test_data.copy()

model = Sequential([
    Dense(64, activation='relu', input_shape = (X_train.shape[1], )), # input layer
    Dense(32, activation='relu'), # hidden layer
    Dense(1, activation='sigmoid') # output layer for binary classification
])

# compile the model
model.compile(optimizer='adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Get model predictions as probabilities
predictions = model.predict(X_test)

# Convert probabilities to binary outcomes (0 or 1)
predicted_classes = (predictions > 0.5).astype("int32").flatten()

test_data10['model_preds'] = predicted_classes
test_data10['probability'] = predictions

# Evaluation
print_metrics(y_test, predicted_classes)

NameError: name 'Sequential' is not defined

In [ ]:
profit_calculation(test_data10)

In [ ]:
test_data10.head(5)

In [ ]:
test_data11 = test_data.copy()

# Train the model
history = model.fit(X_resampled, y_resampled, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Get model predictions as probabilities
predictions1 = model.predict(X_test)

# Convert probabilities to binary outcomes (0 or 1)
predicted_classes1 = (predictions > 0.5).astype("int32").flatten()

test_data11['model_preds'] = predicted_classes1
test_data11['probability'] = predictions1

# Evaluation
print_metrics(y_test, predicted_classes1)

TypeError: LogisticRegression.fit() got an unexpected keyword argument 'epochs'

In [ ]:
profit_calculation(test_data11)

# Play around ith the logistic regression model as that is showing the best profit
- finetune it by playing with penalties and subsets of the data
    - distance buckets, classes
- also try running balanced datasets throught the model.

In [41]:
test_data9 = test_data.copy()

# Define cost matrix
C_FN = 5  # Cost of false negative
C_FP = 1  # Cost of false positive

# Assuming y_train has values 0 and 1
weights = np.ones(y_train.shape[0])
weights[y_train == 1] = C_FN
weights[y_train == 0] = C_FP

# Train logistic regression model with custom weights
model = LogisticRegression()
model.fit(X_train, y_train, sample_weight=weights)

# Predictions
y_predz = model.predict(X_test)

test_data9['model_preds'] = y_predz

# Evaluation
print_metrics(y_test, y_predz)

profit_calculation(test_data9)


Confusion Matrix: 
[[9414 2111]
 [ 862  613]]
Accuracy: 0.7713076923076924
Precision: 0.2250367107195301
Recall: 0.41559322033898305
F1 score: 0.29197427959037864
ROC AUC score: 0.6162130960697084
Total number of bets: 2724
Total return from betting £1.00 on each prediction where model_preds == 1: £138.94
Return per pound invested: £0.05
Model accuracy: 22.50%


# firstly lets try the smote dataset

In [42]:
test_data_smote = test_data.copy()

smote= SMOTE(random_state=42)

X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Define cost matrix
C_FN = 5  # Cost of false negative
C_FP = 1  # Cost of false positive

# Assuming y_train has values 0 and 1
weights = np.ones(y_train_smote.shape[0])
weights[y_train_smote == 1] = C_FN
weights[y_train_smote == 0] = C_FP

# Train logistic regression model with custom weights
model = LogisticRegression()
model.fit(X_train_smote, y_train_smote, sample_weight=weights)

# Predictions
y_pred_smote = model.predict(X_test)

test_data_smote['model_preds'] = y_pred_smote

# Evaluation
print_metrics(y_test, y_pred_smote)

profit_calculation(test_data_smote)


Confusion Matrix: 
[[  443 11082]
 [   13  1462]]
Accuracy: 0.14653846153846153
Precision: 0.11654974489795919
Recall: 0.9911864406779661
F1 score: 0.20857407803694986
ROC AUC score: 0.5148123092760764
Total number of bets: 12544
Total return from betting £1.00 on each prediction where model_preds == 1: £469.40
Return per pound invested: £0.04
Model accuracy: 11.65%


# try random oversampling

In [43]:
# Apply Random Oversampling
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

test_data_ros = test_data.copy()

# Define cost matrix
C_FN = 5  # Cost of false negative
C_FP = 1  # Cost of false positive

# Assuming y_train has values 0 and 1
weights = np.ones(y_resampled.shape[0])
weights[y_resampled == 1] = C_FN
weights[y_resampled == 0] = C_FP

# Train logistic regression model with custom weights
model = LogisticRegression()
model.fit(X_resampled, y_resampled, sample_weight=weights)

# Predictions
y_pred_ros = model.predict(X_test)

test_data_ros['model_preds'] = y_pred_ros

# Evaluation
print_metrics(y_test, y_pred_ros)

profit_calculation(test_data_ros)


Confusion Matrix: 
[[  459 11066]
 [   14  1461]]
Accuracy: 0.1476923076923077
Precision: 0.11662808333998563
Recall: 0.9905084745762712
F1 score: 0.2086844736466219
ROC AUC score: 0.5151674693922571
Total number of bets: 12527
Total return from betting £1.00 on each prediction where model_preds == 1: £459.56
Return per pound invested: £0.04
Model accuracy: 11.66%


# play around with some different penalties

In [44]:
%%script false --no-raise-error

# Copy test data
test_data91 = test_data.copy()

# Define grid of penalty values to test
C_FN_values = range(1,30)  # Range of false negative penalties
C_FP_values = range(1,30)  # Range of false positive penalties

# Initialize variables to store the best results
best_C_FN = None
best_C_FP = None
best_profit = -np.inf  # Start with negative infinity to ensure any positive profit is better

# Iterate over each combination of C_FN and C_FP
for C_FN in C_FN_values:
    for C_FP in C_FP_values:
        # Assign weights based on the current C_FN and C_FP
        weights = np.ones(y_train.shape[0])
        weights[y_train == 1] = C_FN
        weights[y_train == 0] = C_FP

        # Train the logistic regression model
        model = LogisticRegression()
        model.fit(X_train, y_train, sample_weight=weights)

        # Make predictions on the test data
        y_predz = model.predict(X_test)

        # Add predictions to the test data
        test_data91['model_preds'] = y_predz

        # Calculate the profit or loss (assuming profit_calculation function is defined)
        profit_calculation(test_data91)
        

        # Optional: Print progress (for larger grids)
        print(f"C_FN: {C_FN}, C_FP: {C_FP}\n")


# this is the model moving forward

In [45]:
test_data1 = test_data.copy()

# Instantiate and train the Random Forest classifier
params = {
    'n_estimators' : 100,
    'random_state' : 42
}
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    rf_classifier = RandomForestClassifier(**params)
    rf_classifier.fit(X_train, y_train)

    # Log the trained model
    mlflow.sklearn.log_model(rf_classifier, "random_forest_model")

    # Make predictions on the test set
    y_pred = rf_classifier.predict(X_test)

    test_data1['model_preds'] = y_pred

    # Evaluate the model
    # use the eval method
    print_metrics(y_test, y_pred)
    mlflow.end_run()

with open('../models/base_model.pkl', 'wb') as f:
    pickle.dump(rf_classifier, f)

2025/09/10 16:04:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/10 16:04:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Confusion Matrix: 
[[11440    85]
 [ 1468     7]]
Accuracy: 0.8805384615384615
Precision: 0.07608695652173914
Recall: 0.0047457627118644066
F1 score: 0.008934269304403318
ROC AUC score: 0.49868524578109485


In [46]:
test_data_best = test_data.copy()

# Define cost matrix
# we are assigning a greater cost to false negatives.
C_FN = 6  # Cost of false negative
C_FP = 1  # Cost of false positive

# Assuming y_train has values 0 and 1

# creating a numpy array of one's where the length of the array == len(y_train)
# initially every sampple in the dataset is given a value of one
weights = np.ones(y_train.shape[0])

# here we are modifying the weights of all the samples in the training set that belong to class 1
# therefore the weights for samples where the label is 1 are updated to C_FN
weights[y_train == 1] = C_FN

# here we are modifying the weights of all the samples in the training set that belong to class 0
# therefore the weights for samples where the label is 1 are updated to C_FP
weights[y_train == 0] = C_FP

# Train logistic regression model with custom weights
model = LogisticRegression()
model.fit(X_train, y_train, sample_weight=weights)

# Save the model to a file using pickle
with open('../models/logistic_regression_model.pkl', 'wb') as f:
    pickle.dump(model, f)

test_data_best['model_preds'] = y_predz

# Evaluation
print_metrics(y_test, y_predz)
print()
profit_calculation(test_data_best, stake = 1)



Confusion Matrix: 
[[9414 2111]
 [ 862  613]]
Accuracy: 0.7713076923076924
Precision: 0.2250367107195301
Recall: 0.41559322033898305
F1 score: 0.29197427959037864
ROC AUC score: 0.6162130960697084

Total number of bets: 2724
Total return from betting £1.00 on each prediction where model_preds == 1: £138.94
Return per pound invested: £0.05
Model accuracy: 22.50%
